In [ ]:
import os
import glob
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm

In [ ]:
train_path = "/kaggle/input/covid19-image-dataset/Covid19-dataset/train/"

labels = os.listdir(train_path)
print(f"Clases encontradas: {labels}")

Clases encontraddas: ['Normal', 'Viral Pneumonia', 'Covid']


In [ ]:
X = []
y = []

for label_int, label_string in enumerate(labels):
    path = os.path.join(train_path, label_string)
    image_files = glob.glob(path + "/*.jpg") + glob.glob(path + "/*.jpeg") + glob.glob(path + "/*.png")
    
    for img_file in image_files:
        img = cv2.imread(img_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224,224))
        X.append(img)
        y.append(label_int)

X = np.array(X)
y = np.array(y)

X = preprocess_input(X)

print(f"Forma final de X: {X.shape}")

Forma final de X: (251, 244, 224, 3)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Conjunto de entrenamiento: {X_train.shape}")
print(f"Conjunto de validación: {X_val.shape}")

Conjunto de entrenamiento: (200, 244, 224, 3)
Conjunto de validación: (51, 244, 224, 3)


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(labels), activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

/tmp/ipykernel_99/4007518671.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(244,244,3))


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5: None -- [Errno -3] Temporary failure in name resolution

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)

In [ ]:
def get_last_conv_layer_name(model):
    for layer in reversed(base_model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer.name
    return None

def compute_gradcam(model, img_array, pred_index=None, layer_name=None):
    if layer_name is None:
        layer_name = get_last_conv_layer_name(base_model)
        
    last_conv_layer = base_model.get_layer(layer_name)
    
    grad_model = tf.keras.models.Model(
        inputs=[base_model.inputs], 
        outputs=[last_conv_layer.output, base_model.output]
    )
    
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(model.predict(img_array)[0])
        
        class_channel = preds[:, :, :, 0]
        
    grads = tape.gradient(class_channel, last_conv_layer_output)
    
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

def overlay_gradcam(img, heatmap, alpha=0.4):
    img_uint8 = np.uint8(255 * img)
    
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    
    jet_heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    if len(img_uint8.shape) == 2:
        img_uint8 = cv2.cvtColor(img_uint8, cv2.COLOR_GRAY2BGR)
    
    superimposed_img = cv2.addWeighted(img_uint8, 1-alpha, jet_heatmap, alpha, 0)
    
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    
    return superimposed_img

In [ ]:
index = random.randint(0, len(X_val) - 1)
selected_img = X_val[index]
selected_label = y_val[index]

img_input = np.expand_dims(selected_img, axis=0)

prediction = model.predict(img_input)
predicted_class = np.argmax(prediction)

print(f"Etiqueta real: {labels[selected_label]} - Predicción: {labels[predicted_class]} ({np.max(prediction)*100:.2f}%)")

In [ ]:
last_conv_layer = get_last_conv_layer_name(base_model)
print(f"Usando capa convolucional: {last_conv_layer}")

heatmap = compute_gradcam(model, img_input, layer_name=last_conv_layer)

img_visualizable = (selected_img + 1) / 2
superimposed_img = overlay_gradcam(img_visualizable, heatmap)

plt.figure(figsize=(12,5))

plt.subplot(1,3,1)
plt.title("Imagen Original")
plt.imshow(img_visualizable)
plt.axis('off')

plt.subplot(1,3,2)
plt.title("Grad-CAM Heatmap")
plt.imshow(heatmap, cmap='jet')
plt.axis('off')

plt.subplot(1,3,3)
plt.title(f"Superposición\nPredicción: {labels[predicted_class]}")
plt.imshow(superimposed_img)
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def show_multiple_gradcam(model, images, true_labels, num_samples=3):
    indices = random.sample(range(len(images)), min(num_samples, len(images)))
    
    plt.figure(figsize=(15, num_samples*5))
    
    for i, idx in enumerate(indices):
        img = images[idx]
        true_label = true_labels[idx]
        
        img_input = np.expand_dims(img, axis=0)
        
        prediction = model.predict(img_input, verbose=0)
        predicted_class = np.argmax(prediction[0])
        
        last_conv_layer = get_last_conv_layer_name(base_model)
        heatmap = compute_gradcam(model, img_input, layer_name=last_conv_layer)
        
        img_visualizable = (img + 1) / 2
        superimposed_img = overlay_gradcam(img_visualizable, heatmap)
        
        plt.subplot(num_samples, 3, i*3+1)
        plt.title(f"Original\nReal: {labels[true_label]}")
        plt.imshow(img_visualizable)
        plt.axis('off')
        
        plt.subplot(num_samples, 3, i*3+2)
        plt.title(f"Grad-CAM\nPredicción: {labels[predicted_class]}")
        plt.imshow(heatmap, cmap='jet')
        plt.axis('off')
        
        plt.subplot(num_samples, 3, i*3+3)
        plt.title(f"Superposición\nConfianza: {np.max(prediction[0])*100:.2f}%")
        plt.imshow(superimposed_img)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

show_multiple_gradcam(model, X_val, y_val, num_samples=4)

In [ ]:
for class_idx, class_name in enumerate(labels):
    class_indices = [i for i, label in enumerate(y_val) if label == class_idx]
    
    if not class_indices:
        print(f"No hay imágenes de la clase {class_name} en el conjunto de validación")
        continue
    
    sample_indices = random.sample(class_indices, min(2, len(class_indices)))
    class_images = X_val[sample_indices]
    class_labels = y_val[sample_indices]
    
    print(f"\n=== Análisis de la clase: {class_name} ===")
    show_multiple_gradcam(model, class_images, class_labels, num_samples=len(sample_indices))

In [ ]:
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Precisión en validación: {test_acc:.4f}")

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

y_pred = np.argmax(model.predict(X_val), axis=1)

conf_matrix = confusion_matrix(y_val, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
           xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.title('Matriz de Confusión')
plt.show()

print("\nReporte de clasificación:")
print(classification_report(y_val, y_pred, target_names=labels))

In [ ]:
misclassified = [(i, y_val[i], pred) for i, pred in enumerate(y_pred) if pred != y_val[i]]

if misclassified:
    print(f"\nEncontradas {len(misclassified)} imágenes mal clasificadas")
    
    num_to_show = min(3, len(misclassified))
    plt.figure(figsize=(15, num_to_show*5))
    
    for i in range(num_to_show):
        idx, true_label, pred_label = misclassified[i]
        img = X_val[idx]
        
        img_input = np.expand_dims(img, axis=0)
        
        last_conv_layer = get_last_conv_layer_name(base_model)
        
        heatmap_pred = compute_gradcam(model, img_input, pred_index=pred_label, layer_name=last_conv_layer)
        
        heatmap_true = compute_gradcam(model, img_input, pred_index=true_label, layer_name=last_conv_layer)
        
        img_visualizable = (img + 1) / 2
        
        superimposed_pred = overlay_gradcam(img_visualizable, heatmap_pred, alpha=0.5)
        superimposed_true = overlay_gradcam(img_visualizable, heatmap_true, alpha=0.5)
        
        plt.subplot(num_to_show, 4, i*4+1)
        plt.title(f"Original")
        plt.imshow(img_visualizable)
        plt.axis('off')
        
        plt.subplot(num_to_show, 4, i*4+2)
        plt.title(f"Real: {labels[true_label]}")
        plt.imshow(heatmap_true, cmap='jet')
        plt.axis('off')
        
        plt.subplot(num_to_show, 4, i*4+3)
        plt.title(f"Predicho: {labels[pred_label]}")
        plt.imshow(heatmap_pred, cmap='jet')
        plt.axis('off')
        
        plt.subplot(num_to_show, 4, i*4+4)
        plt.title(f"Superposición\n(predicción errónea)")
        plt.imshow(superimposed_pred)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("\nNo se encontraron imágenes mal clasificadas en el conjunto de validación")